In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras.models import Model
from keras import models, layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math
import json
from math import sin, cos, sqrt, atan2,radians

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
  btn = document.getElementById('ok')
  if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
    
setInterval(ClickConnect,60000)'''))


In [ ]:
nameFile = 'ScooterData_example.csv'
dt = pd.read_csv(r'./Dataset/'+nameFile)
f = open(r'./Dataset/grid_info_example.json')
grid_info = json.loads(f.read())
f.close()

data_date_name = dt['start_date']
grid_start_name = dt['starting_grid_id']
data_date_name_test = dt['start_date']
start_day_of_week_test = dt['start_day_of_week']
grid_start_name_test = dt['starting_grid_id']
grid_end_name_test = dt['ending_grid_id']
start_hour_test = dt['start_hour']

data_date_name_test = np.asarray(data_date_name_test.copy())
start_day_of_week_test = np.asarray(start_day_of_week_test.copy())
grid_start_name_test = np.asarray(grid_start_name_test.copy())
grid_end_name_test = np.asarray(grid_end_name_test.copy())
start_hour_test = np.asarray(start_hour_test.copy())

dt['start_day_of_week'] = dt['start_day_of_week'].apply(lambda x: '7' if x[:1] == '0' else x[:1])

h = pd.get_dummies(dt.start_hour, prefix='Hr')
index = 5
for i in h:
  dt.insert(index, i, h[i], True)
  index += 1
d = pd.get_dummies(dt.start_day_of_week)
d = d[['1','2','3','4','5','6','7']]
index = 5
for i in d:
  dt.insert(index, i, d[i], True)
  index += 1

dt['startx'] = dt['startx'].apply(lambda x: float(x)+114)
dt['starty'] = dt['starty'].apply(lambda x: float(x)-51)
dt['endx'] = dt['endx'].apply(lambda x: float(x)+114)
dt['endy'] = dt['endy'].apply(lambda x: float(x)-51)
minX=min([dt['startx'].min(),dt['endx'].min()])
maxX=max([dt['startx'].max(),dt['endx'].max()])
minY=min([dt['starty'].min(),dt['endy'].min()])
maxY=max([dt['starty'].max(),dt['endy'].max()])

dt['startx'] = dt['startx'].apply(lambda x: ((float(x) - minX)/(maxX-minX))*(0.9)+0.1)
dt['starty'] = dt['starty'].apply(lambda x: ((float(x) - minY)/(maxY-minY))*(0.9)+0.1)
dt['endx'] = dt['endx'].apply(lambda x: ((float(x) - minX)/(maxX-minX))*(0.9)+0.1)
dt['endy'] = dt['endy'].apply(lambda x: ((float(x) - minY)/(maxY-minY))*(0.9)+0.1)
dt = dt.drop(columns=['ID','vehicle_type','start_date','trip_duration_min','trip_length_km','starting_grid_id','ending_grid_id','start_hour','start_day_of_week'])

day_table = {
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3,
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
    }

data = np.asarray(dt,dtype=np.float32)
data_cross_size = len(data)/10


In [ ]:
def findkm(x,y):
  R = 6373.0
  lat1 = radians(y[1])
  lon1 = radians(y[0])
  lat2 = radians(x[1])
  lon2 = radians(x[0])

  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
  c = 2 * atan2(sqrt(a), sqrt(1-a))
  return R * c

def findDistance(x,y):
  latitude = (x[0]-y[0])**2
  longitude = (x[1]-y[1])**2
  return math.sqrt(latitude+longitude)

def findNearGrid(lat_long_predict):
    name = None
    min_distance = None
    for key in grid_info:
        temp_distance = findDistance([grid_info[key]["longitude"],grid_info[key]["latitude"]],lat_long_predict)
        if name == None:
            name = key
            min_distance = temp_distance
        elif min_distance > temp_distance:
                name = key
                min_distance = temp_distance
    return name
    
def normalization(d):
  x = list()
  y = list()
  for o in d:
    x.append(o[:33])
    y.append(o[33:])
  x = np.array(x)
  y = np.array(y)
  return x,y

In [ ]:
data_input,data_labels = normalization(data)

In [ ]:
# model = Sequential()
# model.add(Dense(300, activation="relu", input_shape=(33,)))
# model.add(Dropout(0.1))
# model.add(Dense(300, activation="relu"))
# model.add(Dropout(0.1))
# model.add(Dense(300, activation="relu"))
# model.add(Dropout(0.1))
# model.add(Dense(300, activation="relu"))
# model.add(Dropout(0.1))
# model.add(Dense(300, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(2, activation='linear'))
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001 ,name='Adam'), loss='mae')
# model.summary()
model = load_model(r'./Dataset/destination_model_example.h5')
# model.save(r'./Dataset/find_destination_model_example.h5')


In [ ]:
step = int(data_cross_size)
start = 0
end = int(data_cross_size)
process = {
        10000: "1/5",
        20000: "2/5",
        30000: "3/5",
        40000: "4/5"
    }
list_box_real_days = list()
list_box_real_hr = list()
for cross_validation in range(1,11):

  real_days = [[]]*7
  real_days_plot = list()
  real_hr = [[]]*24
  real_hr_plot = list()

  train_input = data_input
  train_labels = data_labels
  if cross_validation != 10:
    test_input = train_input[start:end]
    test_labels = train_labels[start:end]
    start_hour_test_ = start_hour_test[start:end]
    start_day_of_week_test_ = start_day_of_week_test[start:end]
  else:
    test_input = train_input[start:]
    test_labels = train_labels[start:]
    start_hour_test_ = start_hour_test[start:]
    start_day_of_week_test_ = start_day_of_week_test[start:]

  train_input = np.delete(train_input,list(range(start,end)),axis=0)
  train_labels = np.delete(train_labels,list(range(start,end)),axis=0)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  model = load_model(r'/content/drive/My Drive/Colab Notebooks/new_Dataset/destination_model.h5')
  model.fit(train_input, train_labels, epochs=10,callbacks=[callback])

  for op in range(len(test_input)):
    check_ = process.get(op, False)
    if(type(check_) != bool):
      print(f"process grid -> {check_}")
    output = model.predict(test_input[op].reshape(1,-1))
    x_pre = ((output[0][0] - 0.1)/(0.9))*(maxX-minX)+minX-114 # scaleup longitude
    y_pre = ((output[0][1] - 0.1)/(0.9))*(maxY-minY)+minY+51 # scaleup latitude

    near_grid = grid_info[findNearGrid([x_pre, y_pre])]
    x_ac = ((test_labels[op][0] - 0.1)/(0.9))*(maxX-minX)+minX-114 #long
    y_ac = ((test_labels[op][1] - 0.1)/(0.9))*(maxY-minY)+minY+51 #la

    real = findkm([x_ac,y_ac],[near_grid['longitude'],near_grid['latitude']])

    day = start_day_of_week_test_[op][2:]
    if day == "Monday":
      real_days[0].append([real,day])
    elif day == "Tuesday":
      real_days[1].append([real,day])
    elif day == "Wednesday":
      real_days[2].append([real,day])
    elif day == "Thursday":
      real_days[3].append([real,day])
    elif day == "Friday":
      real_days[4].append([real,day])
    elif day == "Saturday":
      real_days[5].append([real,day])
    elif day == "Sunday":
      real_days[6].append([real,day])
    real_hr[start_hour_test_[op]].append([real,start_hour_test_[op]])
  print("process grid -> 5/5 complete")
  start = end
  end += step
  print(f'round {cross_validation}')
  print(f'process... box_plot_data round -> {cross_validation}')

  for k in range(len(real_days)):
    real_days_plot += real_days[k]

  for p in range(len(real_hr)):
    real_hr_plot += real_hr[p]
  list_box_real_days.append(pd.DataFrame(real_days_plot,columns=['km.','days']))
  list_box_real_hr.append(pd.DataFrame(real_hr_plot,columns=['km.','hour']))


In [ ]:
backup_hr = list_box_real_hr.copy()
backup_days = list_box_real_days.copy()

In [ ]:
hour_error = backup_hr[0]
days_error = backup_days[0]
for index1 in range(1,len(backup_hr)):
  hour_error = hour_error.append(backup_hr[index1], ignore_index=True)
for index2 in range(1,len(backup_days)):
  days_error = days_error.append(backup_days[index2], ignore_index=True)

In [ ]:
days_error = days_error.rename(columns={'Distance(km)': 'Distance (km)'})
hour_error =hour_error.rename(columns={'Distance(km)': 'Distance (km)'})

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(12,9)})
ax = sns.boxplot(x="days", y="Distance (km)",order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], data=days_error,palette="Set3",linewidth=1,showfliers = False)
plt.title("Average Distance Days of the week")

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(12,9)})
ax = sns.boxplot(x="hour", y="Distance (km)", data=hour_error,palette="Set3",linewidth=1,showfliers = False)
plt.title("Average Distance hours of the Day ")

In [ ]:
days_error.groupby("days")["km."].describe().to_csv(r'/content/drive/My Drive/Colab Notebooks/new_Dataset/days_of_the_week.csv')
hour_error.groupby("hour")["km."].describe().to_csv(r'/content/drive/My Drive/Colab Notebooks/new_Dataset/hour_of_the_day.csv')

In [ ]:
days_error["km."].describe().to_csv(r'/content/drive/My Drive/Colab Notebooks/new_Dataset/days_of_the_week_v2.csv')
hour_error["km."].describe().to_csv(r'/content/drive/My Drive/Colab Notebooks/new_Dataset/hour_of_the_day_v2.csv')